In [ ]:
%run ../methods/original-sudoku-module.py

In [ ]:
import numpy as np
import pandas as pd
import time
import pickle

# Global variables
N = 3
constraints = get_constraints(N)
weightMatrix = get_weight_matrix(N)

def compute_results(file_name, M, factor):
    results = []
    for _ in range(M):
        sudoku = read_sudoku(file_name)
        sudoku = initialize_sudoku(sudoku, constraints)
        sudoku = solve_sudoku(sudoku,factor,constraints,weightMatrix)
        results.append(sudoku)
    # Save results to a file
    with open('../data/resultaten/results-{:0>3.0f}.pkl'.format(1000*factor), "wb") as f:
        pickle.dump(results, f)  # Using pickle to serialize the results

if __name__ == "__main__":
    start_time = time.time()

    file_names = ["../data/tazuku-01.txt"]
    M = 1
    interval =[704]
    
    for i in interval:
        factor = i / 1000
        compute_results(file_names[0], M, factor)

    print(f"Execution time: {time.time() - start_time} seconds")


In [ ]:
import pickle
interval = list(range(999,1000))
interval = [800,900,990,999]
interval = [990,999]
interval = [704]
results = []

for i in interval:
    solutions = pickle.load(open('../data/resultaten/results-{:0>3.0f}.pkl'.format(i), 'rb'))
    M = len(solutions)
    solution = solutions[0]
    result_array=np.array(solution['result'])
    E = result_array[:,0]
    S = result_array[:,1]
    J = result_array[:,-2]
    t = result_array[:,2]
    results.append([t,E,S,J])    
    score = 0
    for j in range(M):
        score += 1 if solutions[j]['energy']==0 else 0

    print(i, score/M)

In [ ]:
# import matplotlib.pyplot as plt

# # COLORS
# black = np.array([0,0,0])/256
# orange = np.array([230,159,0])/256
# skyblue = np.array([86,180,233])/256
# bluishgreen = np.array([0,158,115])/256
# yellow = np.array([240,228,66])/256
# blue = np.array([0,114,178])/256
# vermillion= np.array([213,94,0])/256
# reddishpurple= np.array([204,121,167])/256

# # SETTINGS

# SMALL_SIZE = 10
# MEDIUM_SIZE = 12
# BIGGER_SIZE = 14

# plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
# plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
# plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
# plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
# plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

X = pd.DataFrame(np.transpose(np.asarray([t,E])),columns=['temperature','Energy'])
Y = X.groupby('temperature').mean()
Z = Y.diff()
idx = Z.idxmax()['Energy']

U = pd.DataFrame(np.transpose(np.asarray([t,S,E])),columns=['T','Entropy','Energy'])
U['t'] = U.diff()['Energy']/U.diff()['Entropy']
U = U[1:]
print(idx)

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True,figsize=(15, 20), dpi=80)

# fig = plt.figure()
# fig, ax =plt.subplots(figsize=(15, 10))
ax = ax1
h1 = ax.plot(Y.index,Y['Energy'],lw=2,label='Energy')
h2 = ax.axvline(x=idx, color=vermillion, lw=1, label='Critical temperature')
ax.axhline(y=0, color="gray", linestyle="dotted")
ax.set_title('ENERGY')
# ax.set_xlabel('Temperature')
ax.set_ylabel('Energy')
ax.legend()

ax = ax2
h1, = ax.plot(Z.index,Z['Energy'],lw=2,label='Heat Capacity')
h2 = ax.axvline(x=idx, color=vermillion, lw=1, label='Critical temperature')
ax.axhline(y=0, color="gray", linestyle="dotted")
ax.set_title('HEAT CAPACITY')
# ax.set_xlabel('Temperature')
ax.set_ylabel('Heat Capacity')
ax.legend()

ax = ax3
h1, = ax.plot(U['T'],U['t'],lw=2,label='Measured vs Steered temperature')
h2 = ax.axvline(x=idx, color=vermillion, lw=1, label='Critical temperature')
ax.axhline(y=0, color="gray", linestyle="dotted")
ax.set_title('TEMPERATURES')
ax.set_xlabel('Steered Temperature')
ax.set_ylabel('Measured Temperature')
ax.set_ylim([-11,11])
ax.legend()

fig.suptitle('ENERGY, HEAT CAPACITY AND CRITICAL TEMPERATURE',y=0.995)
fig.tight_layout() 

# plt.savefig('energy-heatcapacity-diagram.png',transparent=True)